<h1><center>Projet Integre</center></h1>

## <span style="color:yellow">Description des fichiers</span>

### publication (id_publication, date_pub, nbr_publication, article_title) 

author_publication (id_author, id_publication)  
author (id_author, name_author, nbr_publication)

keyword_publication (id_publication, keyword, nbr_use_keyword)  
keyword (keyword, nbr_used)

year_publication (id_publication, id_year)  
year (id_year, year)

publication_venue (id_publication, id_venue)  
venue (id_venue, name_venue)

## <span style="color:yellow">Data Cleaning (bibliothèques re, string), text preprocessing techniques</span>

  * suppression des colonnes inutiles
  *   mettre les titres en minuscule
  *   supprimer la ponctuation
  *   supprimer les valeurs numériques
  *   supprimer les élément qui n'ont aucune signification (balise et autre spéciaux)
  *   tokénisation (uni-gram d'abord)
  *   suppression des mots vides (stop words) 


###  Advanced data cleaning : 

  * racinisation (stemming) / lemmatisation 
  * tokénisation avancée : bi-grams ou tri-grams  
  * gestion des erreurs de frappe ? 



### Organisation des données (sklearn, countvectorizer)
   * corpus (ensemble des titres d'article), 
   * création de la matrice documents-termes (sklearn, countvectorizer)

In [60]:
# option d'affichage pour une meilleur visibilité

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.min_rows', 25)
pd.set_option('display.max_rows', 25)

In [61]:
# Lecture des fichiers csv

author=pd.read_csv('author.csv',sep=',',encoding='cp1252') # (id_author, name_author)
publication=pd.read_csv('publication.csv',sep=',',encoding='cp1252') # Fichier des
del publication['categorie'] # suppression de la colonne qui ne contient que des proceeding
keyword=pd.read_csv('keyword.csv',sep=',',encoding='cp1252')
year=pd.read_csv('year.csv',sep=',',encoding='cp1252')


author_publication=pd.read_csv('publication_author.csv',sep=',',encoding='cp1252')
keyword_publication=pd.read_csv('Publication_keywords.csv',sep=',',encoding='cp1252')
year_publication=pd.read_csv('publication_year.csv',sep=',',encoding='cp1252')

### <span style="color:Pink">Travail sur le fichier publication</span>

#### <span style="color:red">Suppression des doublons et préparation des données avant nettoyage = titre correct des publications</span>

In [62]:
publication[publication.duplicated()] # Doublons sur les lignes

,id_publication,date_pub,nbr_authors,article_title
8205,conf/aabi/LueckmannBKM18,2019-07-12,4,Likelihood-free inference with emulator networks.
52438,conf/birthday/0001ZS18,2018-12-02,3,Network Security Metrics: From Known Vulnerabilities to Zero Day Attacks.
52441,conf/birthday/2017brinksma,2019-05-14,3,ModelEd. TestEd. TrustEd - Essays Dedicated to Ed Brinksma on the Occasion of His 60th Birthday
52443,conf/birthday/2017downey,2019-05-14,6,Computability and Complexity - Essays Dedicated to Rodney G. Downey on the Occasion of His 60th Birthday
52445,conf/birthday/2017larsen,2019-05-14,6,Models. Algorithms. Logics and Tools - Essays Dedicated to Kim Guldstrand Larsen on the Occasion of His 60th Birthday
141352,conf/eusipco/0042HH19,2019-11-25,3,Sensor Selection and Rate Distribution Based Beamforming in Wireless Acoustic Sensor Networks.
334051,conf/konvens/0002W16,2019-11-28,2,A Study on Gaps and Syntactic Boundaries in Spoken Interaction.
367994,conf/nips/2016tsw,2019-05-29,5,Proceedings of the NIPS 2016 Time Series Workshop. co-located with the 30th Annual Conference on Neural Information Processing Systems (NIPS 2016). Barcelona. Spain. December 9. 2016


In [63]:
publication=pd.DataFrame.drop_duplicates(publication) # suppression des doublons

In [64]:
publication[publication['article_title'].duplicated(keep=False)] # doublons aussi sur les colonnes !

# Note : certaines publications de mettre titre ont deux id_publication qui différent d'une lettre et certaines ont deux dates différentes 
# pour un même id_publication

,id_publication,date_pub,nbr_authors,article_title
98,conf/3dim/KimMQ18,2019-07-23,3,Multi-planar Monocular Reconstruction of Manhattan Indoor Scenes.
252,conf/3dor/LangenfeldACCDD18,2019-01-29,23,Protein Shape Retrieval.
275,conf/3dor/SongCCHKLMQSTL17,2017-11-24,11,Protein Shape Retrieval.
572,conf/3pgcic/SabaKIPAAJ17,2019-01-08,7,Home Energy Management Using Social Spider and Bacterial Foraging Algorithm.
997,conf/ACISicis/HaqbeenIAO18,2019-10-19,4,Traffic Adaptive Hybrid MAC with QoS Driven Energy Efficiency for WSNs Through Joint Dynamic Scheduling Mode.
1012,conf/ACISicis/HuX17,2017-07-07,2,A machinima system based on network game.
1013,conf/ACISicis/HuX17a,2017-07-07,2,A machinima system based on network game.
1029,conf/ACISicis/KawaguchiIU19,2019-08-08,3,Learning Neural Circuit by AC Operation and Frequency Signal Output.
1160,conf/ACISicis/WangCZZ17,2019-03-14,4,Real-time image detail enhancement implementing on multi-core DSP platform.
1161,conf/ACISicis/WangCZZ17a,2019-03-14,4,Real-time image detail enhancement implementing on multi-core DSP platform.


In [65]:
delete_rows = publication[publication['article_title'].duplicated(keep='first')]
publication.drop(delete_rows.index,inplace=True)

In [81]:
article_titles = publication["article_title"].str.lower() # articles en minuscle
article_titles = article_titles.tolist() # conversion en liste

#### <span style="color:red">1er nettoyage (suppression des balises et des caractères indésirables et autres aberrations)</span>

In [82]:
import string
import re

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [88]:
# le premier nettoyage consiste à supprimer les caractères et les balises qui ne devraient pas se trouver dans les titres des publications (erreurs d'exportation des données)

# Etant donné la diversité des erreurs une perte d'information est possible 
#   -> ex: on veut supprimer tout ce qui se trouve des parenthèses : on ne sait pas ce qu'il y aura
#   dans la parenthèses. La plus part du temps ce sont des abréviations mais ce peut être un mot clef (bien que rare).. Il faut faire des compromis.

for i in range(len(article_titles)):
    article_titles[i]=re.sub('\[.*?\]','',article_titles[i]) # suppression de toutes les expressions entre crochets
    article_titles[i]=re.sub('\{.*?\}','',article_titles[i]) # suppression de toutes les expressions entre accolades
    article_titles[i]=re.sub('\(.*?\)','',article_titles[i]) # suppression de toutes les expressions entre parenthèses
    article_titles[i]=re.sub('<.+>','',article_titles[i])    # suppression de toute la chaine qui se trouve entre un '<' et un '>' (1ère et dernière occurence)
    article_titles[i]=re.sub('\$.+\$','',article_titles[i])  # suppression de toute la chaine qui se trouve entre un '$' et un '$' (1ère et dernière occurence)
    article_titles[i]=re.sub('\S*[\^,_,&,:,@,#,$,\\\]\S*','',article_titles[i]) # suppression de tous les mots contenant un @ et #.. (@ et # seule inclus aussi)
    article_titles[i]=re.sub('[.,\,,!,?,%,*,§,²,~,\{,\}]','',article_titles[i]) # suppression des caractères spéciaux
    article_titles[i]=re.sub('[+,/]',' ',article_titles[i])
    article_titles[i]=re.sub('-',' ',article_titles[i]) # remplacement des tirets par un espace
    
# reste à gérer les guillemets simples

In [89]:
corpus = article_titles
# reste à supprimer les mois, les pays, les dates...

In [90]:
#article_titles=corpus

In [93]:
matching = [s for s in article_titles if '^' in s]
matching

[]

In [96]:
# suppression des valeurs numériques
for i in range(len(article_titles)):
    article_titles[i]=re.sub('(\s\d+\s|^\d+|\d+$)','',article_titles[i]) # suppression de tous les chiffres qui sont seuls

In [100]:
#matching = [s for s in article_titles if 'france' in s]
#matching

In [101]:
# suppression des mots vides
import gensim
from gensim.parsing.preprocessing import remove_stopwords # pip install -U gensim

In [102]:
# la suppression des mots vides est différente selon les mots clefs utilisés
# dans les différentes librairy. Il est néanmoins possible de modifier cette liste.

all_stopwords = gensim.parsing.preprocessing.STOPWORDS
#print(all_stopwords)

# reste a supprimer les mots qui font une ou deux lettres

In [103]:
for i in range(len(article_titles)):
    article_titles[i]=remove_stopwords(article_titles[i])

In [105]:
#article_titles

#### <span style="color:red">Tokénisation</span>

In [106]:
import nltk # nltk.download('punkt')
from nltk import word_tokenize

In [107]:
for i in range(len(article_titles)):
    article_titles[i] = word_tokenize(article_titles[i])

In [110]:
matching = [s for s in article_titles if '4' in s]
matching

[['packet',
  'scheduler',
  'meeting',
  'deadline',
  'in4',
  ';',
  '4',
  'architecture']]

In [68]:
article_titles

[['complete',
  'estimate',
  'multi-task',
  'approach',
  'depth',
  'completion',
  'monocular',
  'depth',
  'estimation'],
 ['3d',
  'object',
  'discovery',
  'modeling',
  'single',
  'rgb-d',
  'images',
  'containing',
  'multiple',
  'object',
  'instances'],
 ['spatiotemporal',
  'modeling',
  'efficient',
  'registration',
  'dynamic',
  '3d',
  'faces'],
 ['reconstruction',
  'as-is',
  'semantic',
  '3d',
  'models',
  'unorganised',
  'storehouses'],
 ['point',
  'cloud',
  'registration',
  'virtual',
  'points',
  'implicit',
  'quadric',
  'surface',
  'intersections'],
 ['natural', 'language', 'grounded', 'pose', 'forecasting'],
 ['reconstructing',
  '3d',
  'human',
  'poses',
  'keyword',
  'based',
  'image',
  'database',
  'query'],
 ['high',
  'dynamic',
  'range',
  'slam',
  'map-aware',
  'exposure',
  'time',
  'control'],
 ['gravitational', 'approach', 'non-rigid', 'point', 'set', 'registration'],
 ['detailed', 'human', 'avatars', 'monocular', 'video'],
 [